In [1]:
!pip install tensorflow-macos tensorflow-metal opencv-python mediapipe-silicon scikit-learn matplotlib

You should consider upgrading via the '/Users/rakazein/.pyenv/versions/3.8.16/bin/python3.8 -m pip install --upgrade pip' command.


In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

objc[83891]: Class CaptureDelegate is implemented in both /Users/rakazein/.pyenv/versions/3.8.16/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x144f765a0) and /Users/rakazein/.pyenv/versions/3.8.16/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x12f448860). One of the two will be used. Which one is undefined.
objc[83891]: Class CVWindow is implemented in both /Users/rakazein/.pyenv/versions/3.8.16/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x144f765f0) and /Users/rakazein/.pyenv/versions/3.8.16/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x11f650a68). One of the two will be used. Which one is undefined.
objc[83891]: Class CVView is implemented in both /Users/rakazein/.pyenv/versions/3.8.16/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x144f76618) and /Users/rakazein/.pyenv/versions/3.8.16/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x11f650a90). One of the two will be used. Which one i

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [5]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [6]:
def draw_styled_landmarks(image, results):
    #Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [7]:
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [8]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_DATA_V2') 

# Actions that we try to detect
actions = np.array(['a', 'b', 'c', 
                    'd', 'e', 'f', 
                    'g', 'h', 'i', 
                    'j', 'k', 'l', 
                    'm', 'n', 'o', 
                    'p', 'q', 'r', 
                    's', 't', 'u',
                    'v', 'w', 'x',
                    'y', 'z'
                   ])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 0

In [10]:
for action in actions:
    print(os.listdir(os.path.join(DATA_PATH, action)))

['20', '18', '27', '9', '0', '11', '7', '29', '16', '6', '28', '17', '1', '10', '19', '26', '8', '21', '24', '23', '4', '15', '3', '12', '2', '13', '5', '14', '22', '25']
['20', '18', '27', '9', '0', '11', '7', '29', '16', '6', '28', '17', '1', '10', '19', '26', '8', '21', '24', '23', '4', '15', '3', '12', '2', '13', '5', '14', '22', '25']
['20', '18', '27', '9', '0', '11', '7', '29', '16', '6', '28', '17', '1', '10', '19', '26', '8', '21', '24', '23', '4', '15', '3', '12', '2', '13', '5', '14', '22', '25']
['20', '18', '27', '9', '0', '11', '7', '29', '16', '6', '28', '17', '1', '10', '19', '26', '8', '21', '24', '23', '4', '15', '3', '12', '2', '13', '5', '14', '22', '25']
['20', '18', '27', '9', '0', '11', '7', '29', '16', '6', '28', '17', '1', '10', '19', '26', '8', '21', '24', '23', '4', '15', '3', '12', '2', '13', '5', '14', '22', '25']
['20', '18', '27', '9', '0', '11', '7', '29', '16', '6', '28', '17', '1', '10', '19', '26', '8', '21', '24', '23', '4', '15', '3', '12', '2', '13

In [10]:
# for action in actions: 
#     dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
#     for sequence in range(1,no_sequences+1):
#         try: 
#             os.makedirs(os.path.join(DATA_PATH, action, str(dirmax+sequence)))
#         except:
#             pass

In [11]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(start_folder, start_folder+no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                cv2.rectangle(image, (410, 150, 600, 700) , (0, 255, 0), 1)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,50), 
                               cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,50), 
                               cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [ ]:
cap.release()
cv2.destroyAllWindows()